In [1]:
## import modules
from splinter import Browser
import requests
import json
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime as dt
import numpy as np

In [2]:
executable_path = {"executable_path": "chromedriver"}
browser = Browser("chrome", **executable_path, headless=False)

In [3]:
# Build query Url and List of Stocks ticker
base_url = "https://finance.yahoo.com/quote/"
# tickers = ["AAPL", "FB", "IBM", "MSFT"]
# tickers = ['TSLA', 'AAPL', 'ROKU', 'BA', 'ABEV', 'LYFT', 'VALE']
tickers = ['PCG','DIS','CGC','GS','NFLX','UNH','ROKU','AAPL','TSLA','LYFT','BA','UNH','BBBY','CGC','NVDA','MNST','BIDU',
           'NBEV','CF','XLP','BB','VZ','TER','PREV','CCBG','WWE','ADNT','BOOM','PCG','CYBR','CVNA','YETI','PUMP','RDN',
           'VALE','NIO','ABEV','XHR','FCSC','XON','BHVN','ADVM','ACST', 'ADRO','INSY','NOK','NYSE','CCBG',
           'WWE', 'ADNT','BOOM','NOK','PEP','RLGT','DIS','SPY','WUHN','ESNC','GIB','NVA']

In [4]:
# create and initialize list 
close_price_list = []
volume_list = []
mkt_cap_list = []
earnings_Date_list = []
eps_list = []

In [5]:
for ticker in tickers:
    try:
        # Visit Yahoo finance summary page
        url1 = base_url + ticker + '?p=' + ticker + '&.tsrc=fin-srch'
    #     print(url1)
        browser.visit(url1)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")
        summary = soup.find('table', class_="W(100%)").find_all(class_="Trsdu(0.3s) ")
        market_info = soup.find('table', class_="W(100%) M(0) Bdcl(c)").find_all(class_="Trsdu(0.3s) ")
        #print(summary)
        #type(summary)
        close_price = soup.find('div', class_="D(ib) Mend(20px)").text
    #     print(close_price.split('+')[0])
        close_price = summary[0].contents[0]
        close_price_list.append(close_price)
        volume = summary[4].contents[0]
        volume_list.append(volume)
        market_cap = market_info[0].contents[0]
        mkt_cap_list.append(market_cap)
        earnings_Date = market_info[4].contents[0]
        earnings_Date_list.append(earnings_Date)
        eps = market_info[3].contents[0]
        eps_list.append(eps)
    
    except AttributeError:
        pass
    except IndexError:
        pass

In [6]:
# remove B and convert Mkt_cap to Float
new_cap_list = []
for cap in mkt_cap_list:
    try:
        new_cap = cap.replace('B', '')
        new_cap = float(new_cap)
        new_cap_list.append(new_cap)
    except ValueError:
        pass
mkt_cap_list = new_cap_list
print(mkt_cap_list)

[11.079, 238.095, 15.295, 77.875, 157.328, 212.778, 6.421, 961.257, 47.198, 19.697, 214.543, 212.778, 2.302, 15.295, 113.362, 29.982, 59.748, 10.039, 10.82, 5.045, 239.705, 7.76, 7.548, 2.423, 11.079, 4.379, 9.564, 2.739, 2.379, 4.812, 67.973, 4.915, 68.563, 2.39, 2.664, 32.228, 7.548, 2.423, 32.228, 178.47, 238.095, 264.06, 19.383]


In [7]:
# Fix empty date values with a data
new_d_list = []
# print(earnings_Date_list)
# print("/n ---------------- /n")
for d in earnings_Date_list:
    new_d = d.replace('N/A', "2019-02-07")
    new_d_list.append(new_d)
earnings_Date_list = new_d_list
# print(new_d_list)

In [8]:
# remove comma from Volume values
new_vol_list = []
for vol in volume_list:
    try:
        new_vol = vol.replace(',', '')
        new_vol = float(new_vol)
        new_vol_list.append(new_vol)
    except AttributeError:
        pass
volume_list = new_vol_list
# print(volume_list)

In [9]:
# type(earnings_Date_list)

In [10]:
# create and initialize list 
company_name_list = []
company_desc_list = []
company_tel_list = []
company_addr_list = []
key_execs_n_list = []
key_execs_t_list = []
key_execs_s_list = []
url_list = []

In [12]:
for ticker in tickers:
    try:
        # Visit Yahoo finance profile page
        url2 = base_url + ticker + '/profile?p=' + ticker
    #     print(url2)
        browser.visit(url2)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")

        profile1 = soup.find('p', class_="D(ib) W(47.727%) Pend(40px)")
        company_addr = profile1.contents[1] + profile1.contents[5]
        company_addr_list.append(company_addr)
    #     print(company_addr)
        #type(data)
        company_name = soup.find("h3", class_="Fz(m) Mb(10px)").text
        company_name_list.append(company_name)
        company_desc = soup.find('p', class_="Mt(15px) Lh(1.6)").text
        company_desc_list.append(company_desc)
    #     print(company_desc)
        key_execs_data = soup.find('table', class_='W(100%)').find('tbody').find_all('span')
#         print(key_execs_data[0])
        key_execs_name = key_execs_data[0].contents[1]
        key_execs_title = key_execs_data[1].contents[1]
        key_execs_salary = key_execs_data[2].contents[1] 
        if len(key_execs_salary):
            key_execs_salary = 'N/A'
        key_execs_n_list.append(key_execs_name)
        key_execs_t_list.append(key_execs_title)
        key_execs_s_list.append(key_execs_salary)
        profile = soup.find('p', class_="D(ib) W(47.727%) Pend(40px)").find_all('a')
        company_tel = profile[0].text
        url = profile[1].text
        company_tel_list.append(company_tel)
        url_list.append(url)
        
    except IndexError as e:
        pass
    except AttributeError as e:
        pass

In [13]:
#print(key_execs_s_list)

['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']


In [15]:
# convert telephone number to Float
new_tel_list = []
for tel in company_tel_list:
    try:
        new_tel = tel.replace('-', '')
        new_tel = float(new_tel)
        new_tel_list.append(new_tel)
    except ValueError:
        pass
company_tel_list = new_tel_list
#print(company_tel_list)

In [16]:
# remove M and convert Key_Execs_Salary to Float
new_s_list = []
for s in key_execs_s_list:
    try:
        new_s = s.replace('M', '')
        new_s = float(new_s)
        new_s_list.append(new_s)
        
    except ValueError:
        pass
    
key_execs_s_list = new_s_list
# print(key_execs_s_list)

In [17]:
# create and initialize list 
total_ESG_score_list = []
avg_perf_env_list = []
social_score_list = []
governance_score_list = []

In [18]:
for ticker in tickers:
    try:
        # Visit Yahoo finance sustanability page
        url3 = base_url + ticker + '/sustainability?p=' + ticker
    #     print(url3)
        browser.visit(url3)
        # Scrape Financial Data into Soup
        html = browser.html
        soup = bs(html, "html.parser")
        sustain = soup.find_all('div', class_="D(ib) Fz(23px) smartphone_Fz(22px) Fw(600)")
    #     print(sustain[count].contents[0])
    #     count += 1
        total_ESG_score = soup.find('div', class_="Fz(36px) Fw(600) D(ib) Mend(5px)").text
        total_ESG_score_list.append(total_ESG_score)
        avg_perf_env = sustain[0].contents[0]
        avg_perf_env_list.append(avg_perf_env)
        social_score = sustain[1].contents[0]
        social_score_list.append(social_score)
        governance_score = sustain[2].contents[0]
        governance_score_list.append(governance_score)
        
    except AttributeError as e:
        pass

In [20]:
# Create Summary dataframe
data2_dic = {'Ticker':tickers, 'Close_Price':close_price_list, 'Volume':volume_list,'Mkt_Cap':mkt_cap_list, 'EPS':eps_list, 'Earnings_Date':earnings_Date_list}
summary_df = pd.DataFrame.from_dict(data2_dic, orient='index')
summary_df = summary_df.transpose()

In [21]:
summary_df

,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date
0,PCG,21.85,4.65761e+06,11.079,-13.25,2017-09-28
1,DIS,131.75,1.12369e+07,238.095,7.30,2018-12-07
2,CGC,42.85,1.53335e+07,15.295,-1.79,2019-02-07
3,GS,207.90,2.80434e+06,77.875,24.02,2019-05-29
4,NFLX,354.74,8.35316e+06,157.328,2.68,2019-02-07
5,UNH,216.84,1.38491e+07,212.778,12.88,2019-03-08
6,ROKU,56.60,9.48068e+06,6.421,-0.09,2019-02-07
7,AAPL,203.13,2.41958e+07,961.257,12.12,2019-02-08
8,TSLA,271.23,5.87632e+06,47.198,-5.72,2019-02-07
9,LYFT,59.51,6.72887e+06,19.697,-43.04,2019-02-07


In [22]:
# Create Summary dataframe
data_dic = {'Ticker':tickers, 'Company_Name':company_name_list, 'Description':company_desc_list, 'Telephone':company_tel_list, 'Address':company_addr_list,'Key_Execs_Name':key_execs_n_list, 'Key_Execs_Title':key_execs_t_list,'Web_URL':url_list}
profile_df = pd.DataFrame.from_dict(data_dic, orient='index')
profile_df = profile_df.transpose()

In [23]:
profile_df

,Ticker,Company_Name,Description,Telephone,Address,Key_Execs_Name,Key_Execs_Title,Web_URL
0,PCG,PG&E Corporation,"PG&E Corporation, through its subsidiary, Paci...",4.15973e+09,77 Beale StreetPO Box 770000,Mr. Jason P. Wells,Sr. VP & CFO,http://www.pgecorp.com
1,DIS,The Walt Disney Company,"The Walt Disney Company, together with its sub...",8.1856e+09,"500 South Buena Vista StreetBurbank, CA 91521",Mr. Robert A. Iger,Chairman & CEO,http://www.thewaltdisneycompany.com
2,CGC,Canopy Growth Corporation,"Canopy Growth Corporation, together with its s...",2.12902e+09,"1 Hershey DriveSmiths Falls, ON K7A 0A8",Mr. David Michael Solomon,Chairman & CEO,http://www.goldmansachs.com
3,GS,"The Goldman Sachs Group, Inc.","The Goldman Sachs Group, Inc. operates as an i...",4.0854e+09,"200 West StreetNew York, NY 10282",Mr. Wilmot Reed Hastings Jr.,"Co-Founder, Chairman, Pres & CEO",http://www.netflix.com
4,NFLX,"Netflix, Inc.","Netflix, Inc. provides Internet entertainment ...",9.52936e+09,"100 Winchester CircleLos Gatos, CA 95032",Mr. Stephen J. Hemsley,Exec. Chairman,http://www.unitedhealthgroup.com
5,UNH,UnitedHealth Group Incorporated,UnitedHealth Group Incorporated operates as a ...,4.08557e+09,UnitedHealth Group Center9900 Bren Road East,Mr. Anthony J. Wood,"Founder, Chairman, Pres & CEO",http://www.roku.com
6,ROKU,"Roku, Inc.","Roku, Inc. operates a TV streaming platform. T...",4.08996e+09,"150 Winchester CircleLos Gatos, CA 95032",Mr. Timothy D. Cook,CEO & Director,http://www.apple.com
7,AAPL,Apple Inc.,"Apple Inc. designs, manufactures, and markets ...",3.12544e+09,"One Apple Park WayCupertino, CA 95014",Mr. Dennis A. Muilenburg,"Chairman, Pres & CEO",http://www.boeing.com
8,TSLA,"Tesla, Inc.","Tesla, Inc. designs, develops, manufactures, a...",9.52936e+09,"3500 Deer Creek RoadPalo Alto, CA 94304",Mr. Stephen J. Hemsley,Exec. Chairman,http://www.unitedhealthgroup.com
9,LYFT,"Lyft, Inc.","Lyft, Inc. operates a peer-to-peer marketplace...",9.08688e+09,185 Berry StreetSuite 5000,Mr. Warren Eisenberg,Co-Founder & Co-Chairman,http://www.bedbathandbeyond.com


In [24]:
# Create sustainability dataframe
data1_dic = {'Ticker':tickers, 'Total_ESG_Score':total_ESG_score_list, 'AVG_Perf_Env':avg_perf_env_list,'Social_Score':social_score_list, 'Governance_Score':governance_score_list}
sustainability_df = pd.DataFrame.from_dict(data1_dic, orient='index')
sustainability_df = sustainability_df.transpose()

In [25]:
sustainability_df

,Ticker,Total_ESG_Score,AVG_Perf_Env,Social_Score,Governance_Score
0,PCG,70,71,68,71
1,DIS,65,65,63,68
2,CGC,66,88,60,55
3,GS,41,39,41,45
4,NFLX,63,74,53,68
5,UNH,70,89,57,60
6,ROKU,60,70,49,55
7,AAPL,51,54,43,58
8,TSLA,63,74,53,68
9,LYFT,57,59,54,60


In [26]:
# Database 
import pymysql
from sqlalchemy import create_engine
pymysql.install_as_MySQLdb()
rds_connection_string = "root:root123@127.0.0.1:3306/twitter_stocks_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [27]:
engine.table_names()

['profile', 'summary', 'sustainability']

In [28]:
# converted DataFrame into database
summary_df.to_sql(name='summary', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from summary', con=engine).head()

,Ticker,Close_Price,Volume,Mkt_Cap,EPS,Earnings_Date
0,PCG,21.85,4657614.0,11.079,-13.25,2017-09-28
1,DIS,131.75,11236864.0,238.095,7.30,2018-12-07
2,CGC,42.85,15333501.0,15.295,-1.79,2019-02-07
3,GS,207.90,2804336.0,77.875,24.02,2019-05-29
4,NFLX,354.74,8353165.0,157.328,2.68,2019-02-07


In [29]:
# converted DataFrame into database
# pd.to_sql('TRUNCATE TABLE profile', con=engine)
profile_df.to_sql(name='profile', con=engine, if_exists='replace', index=False)

pd.read_sql_query('select * from profile', con=engine).head()

InternalError: (pymysql.err.InternalError) (1366, "Incorrect string value: '\\xC2\\x91n-Ha...' for column 'Description' at row 18") [SQL: 'INSERT INTO profile (`Ticker`, `Company_Name`, `Description`, `Telephone`, `Address`, `Key_Execs_Name`, `Key_Execs_Title`, `Web_URL`) VALUES (%(Ticker)s, %(Company_Name)s, %(Description)s, %(Telephone)s, %(Address)s, %(Key_Execs_Name)s, %(Key_Execs_Title)s, %(Web_URL)s)'] [parameters: ({'Ticker': 'PCG', 'Company_Name': 'PG&E Corporation', 'Description': "PG&E Corporation, through its subsidiary, Pacific Gas and Electric Company, engages in the sale and delivery of electricity and natural gas to reside ... (716 characters truncated) ...  fossil fuel-fired, and solar electricity generation facilities. PG&E Corporation was incorporated in 1995 and is based in San Francisco, California.", 'Telephone': 4159731000.0, 'Address': '77 Beale StreetPO Box 770000', 'Key_Execs_Name': 'Mr. Jason P. Wells', 'Key_Execs_Title': 'Sr. VP & CFO', 'Web_URL': 'http://www

In [30]:
# converted DataFrame into database
sustainability_df.to_sql(name='sustainability', con=engine, if_exists='replace', index=False)
pd.read_sql_query('select * from sustainability', con=engine).head()

,Ticker,Total_ESG_Score,AVG_Perf_Env,Social_Score,Governance_Score
0,PCG,70,71,68,71
1,DIS,65,65,63,68
2,CGC,66,88,60,55
3,GS,41,39,41,45
4,NFLX,63,74,53,68
